## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://github.com/thenameiswin/dualipaGPT/blob/main/dua_lipa_songs.txt

--2023-11-19 14:38:25--  https://github.com/thenameiswin/dualipaGPT/blob/main/dua_lipa_songs.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21504 (21K) [text/plain]
Saving to: ‘dua_lipa_songs.txt’

dua_lipa_songs.txt  100%[===================>]  21.00K  --.-KB/s    in 0.01s   

2023-11-19 14:38:25 (1.46 MB/s) - ‘dua_lipa_songs.txt’ saved [21504/21504]



In [3]:
# read it in to inspect it
with open('dua_lipa_songs.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  21503


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":"dua_lipa_songs.txt","path":"dua_lipa_songs.txt","contentType":"file"}],"totalCount":1}},"fileTreeProcessingTime":1.405471,"foldersToFetch":[],"reducedMotionEnabled":null,"repo":{"id":720764890,"defaultBranch":"main","name":"dualipaGPT","ownerLogin":"thenameiswin","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2023-11-19T14:34:56.000Z","ownerAvatar":"https://avatars.githubusercontent.com/u/96463561?v=4","public":true,"private":false,"isOrgOwned":false},"symbolsExpanded":false,"treeExpanded":true,"refInfo":{"name":"main","listCacheKey":"v0:1700404525.0","canEdit":false,"refType":"branch","currentOid":"f15d86cee7e4db8a8164c31f5b0b8ec3a259db37"},"path":"dua_lipa_songs.txt","currentUser":null,"blob":{"rawLines":["If you don't wanna see me\r","Did a full 180, crazy\r","Thinking 'bout the way I was\r","Did the heartbreak change me? Maybe\r","But look at where I ended up\r","I'm all good already\r

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 "%'(),-./0123456789:=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz{}·
82


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[60, 61, 61, 0, 72, 60, 57, 70, 57]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([21503]) torch.int64
tensor([79,  1, 68, 53, 77, 64, 67, 53, 56,  1, 20, 79,  1, 53, 64, 64, 41, 60,
        67, 70, 72, 55, 73, 72, 71, 27, 66, 53, 54, 64, 57, 56,  1, 20, 58, 53,
        64, 71, 57,  6,  1, 58, 61, 64, 57, 42, 70, 57, 57,  1, 20, 79,  1,  1,
        20, 79,  1, 61, 72, 57, 65, 71,  1, 20, 49, 79,  1, 66, 53, 65, 57,  1,
        20,  1, 56, 73, 53, 52, 64, 61, 68, 53, 52, 71, 67, 66, 59, 71,  8, 72,
        76, 72,  1,  6,  1, 68, 53, 72, 60,  1, 20,  1, 56, 73, 53, 52, 64, 61,
        68, 53, 52, 71, 67, 66, 59, 71,  8, 72, 76, 72,  1,  6,  1, 55, 67, 66,
        72, 57, 66, 72, 42, 77, 68, 57,  1, 20,  1, 58, 61, 64, 57,  1, 80, 51,
         6,  1, 72, 67, 72, 53, 64, 25, 67, 73, 66, 72,  1, 20, 11, 80, 80,  6,
         1, 58, 61, 64, 57, 42, 70, 57, 57, 38, 70, 67, 55, 57, 71, 71, 61, 66,
        59, 42, 61, 65, 57,  1, 20, 11,  8, 14, 10, 15, 14, 17, 11,  6,  1, 58,
        67, 64, 56, 57, 70, 71, 42, 67, 28, 57, 72, 55, 60,  1, 20, 49, 51,  6,
        

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([79,  1, 68, 53, 77, 64, 67, 53, 56])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([79]) the target: 1
when input is tensor([79,  1]) the target: 68
when input is tensor([79,  1, 68]) the target: 53
when input is tensor([79,  1, 68, 53]) the target: 77
when input is tensor([79,  1, 68, 53, 77]) the target: 64
when input is tensor([79,  1, 68, 53, 77, 64]) the target: 67
when input is tensor([79,  1, 68, 53, 77, 64, 67]) the target: 53
when input is tensor([79,  1, 68, 53, 77, 64, 67, 53]) the target: 56


In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[66,  3, 72,  0, 66, 67,  0, 64],
        [66, 59, 71,  8, 72, 76, 72,  1],
        [71, 57, 57,  0, 77, 67, 73,  0],
        [ 0, 11, 18, 10,  6,  0, 55, 70]])
targets:
torch.Size([4, 8])
tensor([[ 3, 72,  0, 66, 67,  0, 64, 57],
        [59, 71,  8, 72, 76, 72,  1,  6],
        [57, 57,  0, 77, 67, 73,  0, 54],
        [11, 18, 10,  6,  0, 55, 70, 53]])
----
when input is [66] the target: 3
when input is [66, 3] the target: 72
when input is [66, 3, 72] the target: 0
when input is [66, 3, 72, 0] the target: 66
when input is [66, 3, 72, 0, 66] the target: 67
when input is [66, 3, 72, 0, 66, 67] the target: 0
when input is [66, 3, 72, 0, 66, 67, 0] the target: 64
when input is [66, 3, 72, 0, 66, 67, 0, 64] the target: 57
when input is [66] the target: 59
when input is [66, 59] the target: 71
when input is [66, 59, 71] the target: 8
when input is [66, 59, 71, 8] the target: 72
when input is [66, 59, 71, 8, 72] the target: 76
when input is [66, 59, 71, 8

In [13]:
print(xb) # our input to the transformer

tensor([[66,  3, 72,  0, 66, 67,  0, 64],
        [66, 59, 71,  8, 72, 76, 72,  1],
        [71, 57, 57,  0, 77, 67, 73,  0],
        [ 0, 11, 18, 10,  6,  0, 55, 70]])


In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 82])
tensor(4.8939, grad_fn=<NllLossBackward0>)
 jF1}jKl5()15[.9j].Zc_}bbUG0Sc1Fr=L=PjrIXGrljfxNA{t\ WGq(}BjGXbq92l,?4 6CAla=Lo-7uGGTYu2 toxC:2V}/thn


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [16]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.237448692321777


In [17]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

 \=uXBBwez)zfX]ToFFa\5BH=dd/kAbzcQ·BwzFa,6o9z)p]vz:_l6cY.z]9A6XvC5f[pyS2aKq43]R}eD8)A6cTGd4C9F{bUcs-vvHKFI,3_a\3/(Udw/D9Xcum{)xP·=hPV,fIIfs /Wt7./t?ZcEv)E yIN.r5"j9dPkU VU7qtaKLo\,R5k4·6oDQr[YIg[],V1\va_v2]RuPZ3iFititpuMuk:_J)-R/5F0laK-JFB3_UywK8ivdGUiWujAeXBWmC%EK8(Q]uT ,%\DfjYIKQu]kZl.th%U9Gh'v1J')Q]hXYPkeSMOYNS?8):NCe75yHYth3XFng23D9uRedFYTZ(CM"i7·{G'vCqNc3q\N/}j6"Y·FR8zMW7[L"hlXFCr·1yyiXBiXR:9A8t[\w=ksyJ.dA6oyF768\SM%}[Mz]MQ??)SuKghf3pyUUd4dpHmLWs.%jOyHz \KTk3quTXdn/I(}WstbU=b1aLZBnzbiQ,. ty 


## The mathematical trick in self-attention

In [18]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [19]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [20]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [21]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [22]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [23]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [24]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [25]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [26]:
k.var()

tensor(1.0449)

In [27]:
q.var()

tensor(1.0700)

In [28]:
wei.var()

tensor(1.0918)

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [30]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [31]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [32]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [33]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [34]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [36]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('dua_lipa_songs.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.211922 M parameters
step 0: train loss 4.5807, val loss 4.5937
step 100: train loss 2.3075, val loss 3.7501
step 200: train loss 2.0510, val loss 3.5675
step 300: train loss 1.9345, val loss 3.3854
step 400: train loss 1.7835, val loss 3.3109
step 500: train loss 1.6079, val loss 3.2650
step 600: train loss 1.4848, val loss 3.1775
step 700: train loss 1.3254, val loss 3.1612
step 800: train loss 1.2192, val loss 3.0671
step 900: train loss 1.1312, val loss 3.1581
step 1000: train loss 1.0198, val loss 3.0288
step 1100: train loss 0.9454, val loss 3.1049
step 1200: train loss 0.8570, val loss 3.0102
step 1300: train loss 0.7832, val loss 3.1252
step 1400: train loss 0.7267, val loss 3.1741
step 1500: train loss 0.6944, val loss 3.2284
step 1600: train loss 0.6275, val loss 3.2758
step 1700: train loss 0.5695, val loss 3.3293
step 1800: train loss 0.5464, val loss 3.4206
step 1900: train loss 0.5199, val loss 3.4024
step 2000: train loss 0.4964, val loss 3.4569
step 2100: train loss 0.